## Raw code dump

from PygorPro which has nicely wrapped functions

In [ ]:
#import everything and set dpi here

from igor.binarywave import load as loadibw
import os

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100 #set dpi of figs for entire notebook

In [ ]:
#absolute path to wave DIRECTORY
wave_dir = r"C:\Users\njana\Desktop\VMHvl Shell Inputs\#2_Dec23\201222M#1049\Dec2022A\Dec2022A_Waves"
#load every wave into a dictionary
waves = {}
for filename in os.listdir(wave_dir):
    #print(filename)
    waves[filename[filename.find("_")+1:-4]] = loadibw(wave_dir+"/"+str(filename))['wave']['wData'] #keeps only the waves
#convert into a nice structured dataframe
wavedata = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in waves.items()]))
#is this even necessary? maybe to save into an addressable HDF5 file or with nwb format?

#digitization frequency (samples per millisecond)
freq = 50 #per ms (50 KHz)

# Gets asc path from wave_dir
cellname = wave_dir[wave_dir.rfind("\\")+1:wave_dir.rfind("_")]
asc_dir = wave_dir[:wave_dir.rfind("\\")+1]+cellname+".asc"

#reads paramdata into df
rawdata = pd.read_csv(asc_dir, encoding= 'unicode_escape', names =  ["Header", "Recording Mode","Amplifier","Data Kind","Time Offset","Y-unit","X-unit","X-interval","X-start","Y-Offset","Y-range","Holding","Pipette Resistance","Seal Resistance","C-slow","R-series","Rs-value","Leak Comp.","M-conductance"])
paramdata = rawdata[rawdata['Header'].str.contains('TRACE')][["Header","C-slow","R-series"]]
paramdata["Header"] = paramdata["Header"].str.replace("TRACE_","")

In [ ]:
#the perfect test plot for single pair of traces (stim and recorded channel)
# DO NOT DELETE THE ORIGINAL PERFECTION

freq = 50 # in KHz or samples per ms, both are the same.

stim = waves['1_2_4_1']
rec = waves['1_2_4_2']
time = 0.001*np.arange(1,stim.shape[0]+1)/freq

sns.set_theme()
plt.subplots(2,1,gridspec_kw={'height_ratios': [1, 4]})
plt.subplots_adjust(hspace=0)


#plot stim
plt.subplot(2,1,1)
plt.plot(time, stim, color = 'b')
plt.text(time[-1],np.max(stim),"5 ms pulse", horizontalalignment='right', verticalalignment='top', color = 'b')
plt.gca().set_xticks([])
plt.gca().set_yticks([])

#plot recorded channel
plt.subplot(2,1,2)
plt.plot(time, rec*1e9, color = 'r')
plt.text(time[-1],np.min(rec*1e9),"-70 mV holding", horizontalalignment='right', verticalalignment='bottom', color = 'r')
plt.ylabel("pA")
plt.xlabel("s")
#plt.gca().set_xticks([])

plt.gcf().set_size_inches(10,4)

In [ ]:
#Plot everything together stupidly.

sns.set_theme()
plt.subplots(2,1,gridspec_kw={'height_ratios': [1, 4]})
plt.subplots_adjust(hspace=0)

#plot stim
plt.subplot(2,1,1)

for ele in waves.keys():
    if ele[-1] == "1":
        plt.plot(waves[ele], color = 'b')

plt.gca().set_xticks([])
plt.gca().set_yticks([])

#plot recorded channel
plt.subplot(2,1,2)

for ele in waves.keys():
    if ele[-1] == "2":
        plt.plot(waves[ele]*1e9, color = 'r', linewidth=0.1, alpha = 1)
plt.gca().set_xticks([])

plt.gcf().set_size_inches(10,4)

In [ ]:
# IMPORTANT/URGENT: determine max value around stim window for each wave, plot for the cell, store data across cells.
'''

Finds all peaks!

now to wrap it into a nice function that accepts two waves.

'''
stim = waves['1_2_1_1']
rec = waves['1_2_1_2']*1e9

t = 50 #time window duration from stim start

#find stim starts:

stim_locs = []
for i in enumerate(stim):
    i = i[0]
    if stim[i-1] < 1 and stim[i] > 1:
        stim_locs += [i]
        
baseline = np.mean(rec[:stim_locs[0]])
peaks = []

for i in stim_locs:
    peaks += [np.min(rec[i:i+t*freq])]
    
peaks = np.array(peaks)
amps = peaks - baseline

In [ ]:
# protocol identifier/sorter

'''

Identifies protocols based on number of sweeps,

1  > 20 Train
3  > Test Pulse (single)
10 > Paired Pulse Protocol

should be extended on other parameters(?)

'''

wavlst = list(waves.keys())

train_20p = []
one_pulse = []
two_pulse = []
unassigned = []
dump = []

# loop through the thingy
for ele in wavlst:
    
    if len(dump) == 0: # if dump is empty, just add the element
        dump += [ele]
        
    else: #if dump is not empty, check if it's from the same series as previous
        
        if ele[2:4] == dump[-1][2:4]: #if same as previous, add to dump
            dump += [ele]
            
        elif len(dump) == 20: #if not same as previous and dump has 10, put it to two pulse
            two_pulse += [dump]
            dump = [ele]
            
        elif len(dump) == 6: # if not same as previous and not 10, but 3, one pulse
            one_pulse += [dump]
            dump = [ele]
            
        elif len(dump) == 2: # if not same as previous and not 10 or 3, train 20 pulses
            train_20p += [dump]
            dump = [ele]
            
        else: # if none of the above, put to unassigned
            unassigned += [dump]
            dump = [ele]

# do something about sweeps remaining in dump at the end of the for loop
    
if len(dump) == 0: #if same as previous, add to dump
    dump = []       
elif len(dump) == 20: #if not same as previous and dump has 10, put it to two pulse
    two_pulse += [dump]
    dump = []      
elif len(dump) == 6: # if not same as previous and not 10, but 3, one pulse
    one_pulse += [dump]
    dump = [] 
elif len(dump) == 2: # if not same as previous and not 10 or 3, train 20 pulses
    train_20p += [dump]
    dump = []       
else: # if none of the above, put to unassigned
    unassigned += [dump]
    dump = []
